# Class Practice Session

## Goal:
- OpenGL Rendering using vertex buffer and index buffer
- Using Assimp reader to read and render models in .gltf format

## Change from Assignment 6
- Replace Obj Reader with Assimp reader.
  - We will import assimp_py
  - We will use the helper file for loading model data: "from loadModelUsingAssimp_V1 import getObjectDataList"
  - "getObjectDataList" takes a file (filepath) as the first parameter and an optional verbose value as True/False for verbose output. getObjectDataList returns a number of values. They are: geomDataList, indexDataList, bounds, texNames, scene. The scene file is needed for traversal of the Hierarchical part of the scene graph.
- Use model transformations in vertex shader in addition to view and perspective transformations.
- Create a list of renderables and create an associated list of texture files
  - Renderable using indexData needs additional parameters to the ctx.vertex_array method. For ex: if gl is the ctx then you will use gl.vertex_array(program,(program,[(geomBuffer, "3f 2f", "position", "uv")],index_buffer=indexBuffer,index_element_size=4) to create a renderable.
- Recursive traversal of the hierarchical part of the scene graph for rendering

## Imports

In [1]:
import pygame
import moderngl
import numpy
import glm

pygame 2.6.1 (SDL 2.28.4, Python 3.10.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Intialize

In [2]:
pygame.init() # Initlizes its different modules. Display module is one of them.

width = 840
height = 480
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE) 
pygame.display.set_mode((width, height), flags= pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE)
pygame.display.set_caption(title = "Class Practice: Instructor")
gl = moderngl.get_context() # Get Previously created context.
gl.info["GL_VERSION"]

'4.6.0 - Build 32.0.101.6881'

## Create shader program(s) and Renderables

#### Shaders

In [3]:
#
# Vertex shader(s)
#
vertex_shader_code = '''
#version 330 core
layout (location=0) in vec3 position;
layout (location=1) in vec2 uv;

uniform mat4 model, view, perspective;
out vec2 f_uv; // Texture coordinate

void main() {
    f_uv = uv;
    vec4 P = perspective*view*model*vec4(position, 1);
    gl_Position = P;
}
'''

#
# Fragment shader(s)
#
fragment_shader_code = '''
#version 330 core
in vec2 f_uv;
uniform sampler2D map;
// Add output variable here
out vec4 color;

void main() {
    //set output color value here
    color = texture(map, f_uv);
}
'''
#
# Programs
#

program = gl.program(
    vertex_shader= vertex_shader_code,
    fragment_shader= fragment_shader_code
)

### Renderable(s): Buffer(s), Vertex Array

#### We will revisit our Clock face example where we wanted a square (quad) clockface whoses position and texture coordinates of each vertex are as follows:
```
    -1, -1, 0,   0, 0, # bottom left corner  
     1, -1, 0,   1, 0, # bottom right corner  
     1,  1, 0,   1, 1, # top right corner
    -1,  1, 0,   0, 1  # top left corner
```
But we ended up using 6 vertices to render two arrays. Duplicating two of the vertices. The index of the vertices are as follow
```
    0, 1, 2, # For the first triangle
    2, 3, 0  # for the second triangle
```
We will remove this redundancy by using only 4 verices, but create the triangle faces by providing indices of the vertex array. That would requie us to load the index array (numpy array of int4 type to a buffer and add the index buffer to the vertex_array to create the renderable. For ex: gl.vertex_array(program,(program,[(geomBuffer, "3f 2f", "position", "uv")],index_buffer=indexBuffer,index_element_size=4)

In [4]:
geomData = numpy.array([
    -1, -1, 0,   0, 0, # bottom left corner
     1, -1, 0,   1, 0, # bottom right corner
     1,  1, 0,   1, 1, # top right corner
     1,  1, 0,   1, 1, # top right corner
    -1,  1, 0,   0, 1,  # top left corner   
    -1, -1, 0,   0, 0, # bottom left corner
]).astype("float32")

class bound:
    bounding_box = [[-1,-1,0], [1, 1, 0]]
    center = [0,0,0]
    radius = 1.414

geomBuffer = gl.buffer(geomData)
renderable = gl.vertex_array(program,
    [(geomBuffer, "3f 2f", "position", "uv")]
)

### Create Texture object and specify filter and wrap options.

In [5]:
texture_img = pygame.image.load("clockFace.png") 
texture_data = pygame.image.tobytes(texture_img,"RGB", True) 
texture = gl.texture(texture_img.get_size(), data = texture_data, components=3)
texture.build_mipmaps()
sampler = gl.sampler(texture=texture, filter=(gl.LINEAR_MIPMAP_LINEAR, gl.LINEAR), repeat_x = True, repeat_y = True)
sampler.use(0)

### Define Camera Parameters

In [6]:
displacement_vector = 2*bound.radius*glm.vec3(0,0,1) #glm.rotate(glm.vec3(0,1,0), glm.radians(60), glm.vec3(1,0,0)) #
    
target_point = glm.vec3(bound.center)
up_vector = glm.vec3(0,1,0)

### View volume parameters
fov_radian = glm.radians(45) # In radian
aspect = width/height
near = bound.radius
far = 3*bound.radius
perspectiveMatrix = glm.perspective(fov_radian, aspect, near, far)

### Render loop

In [7]:
running = True
clock = pygame.time.Clock()
alpha = 0
pause = True
gl.enable(gl.DEPTH_TEST)
while running:   
    # poll for events
    # pygame.QUIT event means the user clicked X to close your window
    # event.key == 27 means Escape key is pressed.
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif (event.type ==  pygame.KEYDOWN):
            if  event.key == 27:
                running = False
            elif event.key == pygame.K_p:
                pause = not pause
        elif (event.type == pygame.WINDOWRESIZED):
            width = event.x
            height = event.y
            perspectiveMatrix = glm.perspective(fov_radian, width/height, near, far)

    # create the aspect ratio correction matrix
    
    eye_point = glm.vec3(bound.center) + glm.rotate(displacement_vector, glm.radians(alpha), glm.vec3(0,1,0))
    viewMatrix = glm.lookAt(eye_point, target_point, up_vector)

    
    ### Add render code below
    # Clear the display window with the specified R, G, B values using function ctx.clear(R, G, B)
    gl.clear(0.5,0.5,0.0)
    
    # Make one or more Render Calls to instruct the GPU to render by executing the shader program with the provided data.
    program["view"].write(viewMatrix)
    program["perspective"].write(perspectiveMatrix)
    
    program["map"] = 0
    program["model"].write(glm.mat4(1))
    renderable.render()

    pygame.display.flip()
    if not pause:
        clock.tick(60)  # limits FPS to 10
        alpha = alpha + 1
        if alpha > 360:
            alpha = 0
    
pygame.display.quit()

### Assimp data structure
**scene:**
- meshes: array of mesh objects
- materials: array of material objects
- rootnode: node

**mesh:**
- name
- vertices: array of vertex positions in 3D
- texturecoords: a list of texture coordinate arrays! (we will now use the first array)
- normals: array of normal vectors in 3D (to be used later)
- face indices
- index to material array

**material""
- Reflection properties
- Textures

**node:**
- name
- transformation: 4x4 transformation that must be applied at this node
- mesh indices: an array of 0 or more indices to the mesh objects defined in the scene meshes
- children: array of 0 or more child nodes

## Recursive Render